In [1]:
import numpy as np                   
import pandas as pd                  
import matplotlib.pyplot as plt
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Read the CSV file into a DataFrame with header=None
data = pd.read_csv("filtered_file duplicate.csv", header=None)

# Extract the first row as column names and remove it from the DataFrame
column_names = data.iloc[0]
data = data[1:]

# Set the column names to the DataFrame
data.columns = column_names
data.head(5)

,selected_text,sentiment
1,"I`d have responded, if I were going",3
2,Sooo SAD,1
3,bullying me,1
4,leave me alone,1
5,"Sons of bitches,",1


In [3]:
data2 = pd.read_csv("Apple-Twitter-Sentiment-DFE.csv", sep=';', encoding='utf-8', header = 0)
headers = data2.iloc[0]
data2.columns = headers
data2 = data2.iloc[1:]
filtered_data = pd.DataFrame()
filtered_data['selected_text'] = data2['text']
filtered_data['sentiment'] = data2['sentiment']
filtered_data.head(5)

,selected_text,sentiment
1,#AAPL:The 10 best Steve Jobs emails ever...htt...,3
2,RT @JPDesloges: Why AAPL Stock Had a Mini-Flas...,3
3,My cat only chews @apple cords. Such an #Apple...,3
4,I agree with @jimcramer that the #IndividualIn...,3
5,Nobody expects the Spanish Inquisition #AAPL,3


In [4]:
#combine the two data sets apple + general
data = pd.concat([data, filtered_data], axis=0)
data.head(100000)

,selected_text,sentiment
1,"I`d have responded, if I were going",3
2,Sooo SAD,1
3,bullying me,1
4,leave me alone,1
5,"Sons of bitches,",1
...,...,...
3882,(Via FC) Apple Is Warming Up To Social Media -...,3
3883,RT @MMLXIV: there is no avocado emoji may I as...,3
3884,@marcbulandr I could not agree more. Between @...,5
3885,My iPhone 5's photos are no longer downloading...,1
